In [77]:
#!pip install transformers
#!pip install sentencepiece

In [78]:
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


In [79]:
import os
import pickle
import argparse
import unicodedata
import pandas as pd

# Import Tokenizer
from transformers import AutoTokenizer
from kobert_tokenizer import KoBERTTokenizer

python ./preprocess/encoding.py --load_fn data/dataset/train.pickle --save_path data/encoded


**argparser에서 --load_fn 을 "data/dataset/train.pickle"로 한번 실행 후**
**"data/dataset/test.pickle"로 변경하여 다시 실행시켜야 합니다.**

In [ ]:
def define_argparser():
    p = argparse.ArgumentParser()
    # skt/kobert-base-v1
    p.add_argument("--pretrained_model_name", default ='klue/roberta-base', help="Pre-trained model name to be going to train. Tokenizer will be assigned based on the model.")
    p.add_argument("--load_fn", default='data/dataset/train.pickle', help="Original data to be going to preprocess.")
    #p.add_argument("--load_fn", default='data/dataset/test.pickle', help="Original data to be going to preprocess.")

    p.add_argument("--save_path", default='data/encoded', help="Original data to be going to preprocess.")
    p.add_argument("--with_text", action="store_true", help="Return values with text for inference.")

    config = p.parse_args(args=[])

    return config

In [81]:
config = define_argparser()
config

Namespace(pretrained_model_name='klue/roberta-base', load_fn='data/dataset/test.pickle', save_path='data/encoded', with_text=False)

In [82]:
USE_KOBERT = True if config.pretrained_model_name == 'skt/kobert-base-v1' else False
data = pd.read_pickle(config.load_fn)
texts = data['sentence'].values.tolist()
nes = data['ne'].values.tolist()

In [83]:
data

,sentence,ne,source,sentence_class
45,본적 경북 의성군 의성읍 경북대로 5690-25,"{1: {'form': '본적', 'label': 'LC_DOMICILE', 'be...",None,LC
172,한적한 길을 지나가는 여아를 상대로 나일론 줄로 목을 졸라 실신시킨 후 성추행을 하...,"{1: {'form': '길', 'label': 'LC_OTHER_FACILITY'...",None,GD
192,피고인은 위 경찰관들이 위 정우식의 모발 채취를 하지 못하도록 하기 위하여 위 정우...,"{1: {'form': '피고인', 'label': 'PS_ASSAILANT', '...",None,JB
210,이 사건 당일 필로폰 투약 혐의로 수사를 받고 있던 친구인 정우식의 도주(모발 검사...,"{1: {'form': '당일', 'label': 'TI_SUBSTITUTE', '...",None,QT
107,주거 대구 달성군 가창면 가창로 169,"{1: {'form': '주거', 'label': 'LC_RESIDENCE', 'b...",None,LC
156,"나. 판시 제6항 기재 범행에 관하여, 피고인이 피해자 지화연의 목을 조르자 피해자...","{1: {'form': '피고인', 'label': 'PS_ASSAILANT', '...",None,PC
303,"이러한 사정도 임조은이 피고인과 합의하에 성관계를 맺었다면 있을 수 없는 것인 점,...","{1: {'form': '임조은', 'label': 'NM', 'begin': 8,...",None,RL
62,원심의 증거조사 결과와 항소심 변론종결시까지 추가로 이루어진 증거조사 결과를 종합하...,"{1: {'form': '증인', 'label': 'PS_OTHERS', 'begi...",None,PS
212,이러한 행위는 통상적으로 빈번하게 일어나고 있는 공무집행방해의 형태 즉 술에 취해 ...,"{1: {'form': '공무집행방해', 'label': 'NC', 'begin':...",None,DP
208,양형의 이유,{},None,Out


In [84]:
texts

['본적 경북 의성군 의성읍 경북대로 5690-25',
 '한적한 길을 지나가는 여아를 상대로 나일론 줄로 목을 졸라 실신시킨 후 성추행을 하였고, 심지어는 자신의 처가 교사로 근무하는 초등학교의 학생의 집에 침입하여 부녀자를 희롱하려다 미수에 그치고도 그 후 다시 그 집에 침입하여 여아를 추행하는 등 그 죄질이 아주 좋지 않고,',
 '피고인은 위 경찰관들이 위 정우식의 모발 채취를 하지 못하도록 하기 위하여 위 정우식을 막고 서 있는 위 최성욱의 팔을 잡아당기고, 위 김주하의 팔과 상의를 잡아 뒤로 밀어 넘어뜨리고, 그 틈을 이용하여 도주하는 위 정우식을 위 김주하가 뒤따라가자,',
 '이 사건 당일 필로폰 투약 혐의로 수사를 받고 있던 친구인 정우식의 도주(모발 검사의 회피)를 도울 목적으로 법원이 발부한 압수수색영장에 의하여 정당한 공무집행 중인 경찰관 2명에게 폭력을 행사함으로써 경찰수사에 큰 차질이 생기게 된 점',
 '주거 대구 달성군 가창면 가창로 169',
 '나. 판시 제6항 기재 범행에 관하여, 피고인이 피해자 지화연의 목을 조르자 피해자가 실신하는 바람에 피고인이 너무 놀라 피해자를 운동장 바닥에 내려놓고 가버린 사실이 있을 뿐 지화연의 가슴과 음부를 만진 사실은 없다.',
 "이러한 사정도 임조은이 피고인과 합의하에 성관계를 맺었다면 있을 수 없는 것인 점, ④ 검거 당시 피고인은 권한별이 자신의 여자친구라고 하면서도 '여자 친구의 이름을 대라'는 류현민의 요구에 대답을 하지 못하였던 점, ⑤ 권한별이 피고인을 모함할 아무런 동기를 찾아볼 수 없는 점 등을 종합하여 보면,",
 '원심의 증거조사 결과와 항소심 변론종결시까지 추가로 이루어진 증거조사 결과를 종합하면 원심 증인이 한 진술의 신빙성 유무에 대한 원심의 판단을 그대로 유지하는 것이 현저히 부당하다고 인정되는 예외적인 경우가 아니라면,',
 '이러한 행위는 통상적으로 빈번하게 일어나고 있는 공무집행방해의 형태 즉 술에 취해 우발적으로 범하는 단순한 공무집행방해 등과는 달리 공권력

In [85]:
nes

[{1: {'form': '본적', 'label': 'LC_DOMICILE', 'begin': 0, 'end': 2},
  2: {'form': '경북 의성군 의성읍 경북대로 5690-25',
   'label': 'LC_REGION',
   'begin': 3,
   'end': 26}},
 {1: {'form': '길', 'label': 'LC_OTHER_FACILITY', 'begin': 4, 'end': 5},
  2: {'form': '여아', 'label': 'GD', 'begin': 12, 'end': 14},
  3: {'form': '나일론 줄', 'label': 'EQ_BIND', 'begin': 20, 'end': 25},
  4: {'form': '목', 'label': 'BP', 'begin': 27, 'end': 28},
  5: {'form': '실신', 'label': 'PC', 'begin': 33, 'end': 35},
  6: {'form': '성추행', 'label': 'NC', 'begin': 40, 'end': 43},
  7: {'form': '자신', 'label': 'RL', 'begin': 55, 'end': 57},
  8: {'form': '처', 'label': 'RL', 'begin': 59, 'end': 60},
  9: {'form': '교사', 'label': 'JB', 'begin': 62, 'end': 64},
  10: {'form': '초등학교', 'label': 'LC_OTHER_FACILITY', 'begin': 71, 'end': 75},
  11: {'form': '집',
   'label': 'LC_RESIDENTIAL_FACILITY',
   'begin': 81,
   'end': 82},
  12: {'form': '희롱', 'label': 'NC', 'begin': 94, 'end': 96},
  13: {'form': '집', 'label': 'RL', 'begin': 118,

In [86]:
pretrained_model_name = config.pretrained_model_name
print(pretrained_model_name)

klue/roberta-base


In [87]:
if USE_KOBERT:
    tokenizer_loader = KoBERTTokenizer # 해당 안 됨
else:
    tokenizer_loader = AutoTokenizer

tokenizer = tokenizer_loader.from_pretrained(pretrained_model_name)
print("Tokenizer loaded :", tokenizer.name_or_path)

Tokenizer loaded : klue/roberta-base


In [88]:
def get_label_dict(labels):
    BIO_labels = ['O']
    for label in labels:
        BIO_labels.append('B-'+label)
        BIO_labels.append('I-'+label)

    label_to_index = {label: index for index, label in enumerate(BIO_labels)}
    index_to_label = {index: label for index, label in enumerate(BIO_labels)}

    return label_to_index, index_to_label

In [89]:
# label_list = ["PS", "FD", "TR", "AF", "OG", "LC", "CV", "DT", "TI", "QT", "EV", "AM", "PT", "MT", "TM"]
label_list = ["CN", "NC", "RL", "LC", "NM", "GD", "AG", "JB", "PS", "DP", "TI", "MC", "PC", "BP", "QT", "RA", "EQ"]
label_to_index, index_to_label = get_label_dict(label_list) # 레이블 리스트를 받아서 BIO 매핑을 진행
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
max_length = 512

In [90]:
print(label_to_index)
print(index_to_label)
print(cls_token)
print(sep_token)

{'O': 0, 'B-CN': 1, 'I-CN': 2, 'B-NC': 3, 'I-NC': 4, 'B-RL': 5, 'I-RL': 6, 'B-LC': 7, 'I-LC': 8, 'B-NM': 9, 'I-NM': 10, 'B-GD': 11, 'I-GD': 12, 'B-AG': 13, 'I-AG': 14, 'B-JB': 15, 'I-JB': 16, 'B-PS': 17, 'I-PS': 18, 'B-DP': 19, 'I-DP': 20, 'B-TI': 21, 'I-TI': 22, 'B-MC': 23, 'I-MC': 24, 'B-PC': 25, 'I-PC': 26, 'B-BP': 27, 'I-BP': 28, 'B-QT': 29, 'I-QT': 30, 'B-RA': 31, 'I-RA': 32, 'B-EQ': 33, 'I-EQ': 34}
{0: 'O', 1: 'B-CN', 2: 'I-CN', 3: 'B-NC', 4: 'I-NC', 5: 'B-RL', 6: 'I-RL', 7: 'B-LC', 8: 'I-LC', 9: 'B-NM', 10: 'I-NM', 11: 'B-GD', 12: 'I-GD', 13: 'B-AG', 14: 'I-AG', 15: 'B-JB', 16: 'I-JB', 17: 'B-PS', 18: 'I-PS', 19: 'B-DP', 20: 'I-DP', 21: 'B-TI', 22: 'I-TI', 23: 'B-MC', 24: 'I-MC', 25: 'B-PC', 26: 'I-PC', 27: 'B-BP', 28: 'I-BP', 29: 'B-QT', 30: 'I-QT', 31: 'B-RA', 32: 'I-RA', 33: 'B-EQ', 34: 'I-EQ'}
[CLS]
[SEP]


In [91]:
if USE_KOBERT:
    encoded = tokenizer(texts,
                        add_special_tokens=True,
                        padding=False,
                        return_attention_mask=True,
                        truncation=True,
                        max_length=max_length)
else:
    encoded = tokenizer(texts,
                        add_special_tokens=True,
                        padding=False,
                        return_attention_mask=True,
                        truncation=True,
                        max_length=max_length,
                        return_offsets_mapping=True)

print("Sentences encoded : |input_ids| %d, |attention_mask| %d" %
        (len(encoded['input_ids']), len((encoded['attention_mask']))))

Sentences encoded : |input_ids| 48, |attention_mask| 48


In [92]:
print(type(encoded))
print(dir(encoded))

<class 'transformers.tokenization_utils_base.BatchEncoding'>
['_MutableMapping__marker', '__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_encodings', '_n_sequences', 'char_to_token', 'char_to_word', 'clear', 'convert_to_tensors', 'copy', 'data', 'encodings', 'fromkeys', 'get', 'is_fast', 'items', 'keys', 'n_sequences', 'pop', 'popitem', 'sequence_ids', 'setdefault', 'to', 'token_to_chars', 'token_to_sequence', 'token_to_word', 'tokens', '

In [93]:
for i in range (0, 1):
    print(encoded['input_ids'][i])  # 토큰 ID 리스트
    print(encoded['attention_mask'][i])  # 어텐션 마스크
    print(encoded['token_type_ids'][i])  # (있을 경우) 토큰 타입 ID
    print(encoded['offset_mapping'][i])  # 오프셋 매핑
    print(len(encoded['input_ids'][i]))

[0, 1163, 2125, 4724, 15211, 2242, 15211, 2058, 4724, 3770, 6086, 11445, 17, 3912, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(0, 0), (0, 1), (1, 2), (3, 5), (6, 8), (8, 9), (10, 12), (12, 13), (14, 16), (16, 18), (19, 21), (21, 23), (23, 24), (24, 26), (0, 0)]
15


In [94]:
df = pd.DataFrame([encoded["input_ids"], encoded["attention_mask"], data['sentence_class'].values],
                  index=["input_ids", "attention_mask", "sentence_class"]).T

In [95]:
df

,input_ids,attention_mask,sentence_class
0,"[0, 1163, 2125, 4724, 15211, 2242, 15211, 2058...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",LC
1,"[0, 19243, 2470, 647, 2069, 7129, 2259, 29001,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",GD
2,"[0, 9679, 2073, 1485, 9358, 7285, 1485, 12475,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",JB
3,"[0, 1504, 3863, 7494, 23252, 2357, 22133, 4456...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",QT
4,"[0, 5782, 3900, 17429, 19513, 2460, 19513, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",LC
5,"[0, 717, 18, 20928, 1545, 2248, 2421, 7475, 81...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",PC
6,"[0, 3839, 2470, 5183, 2119, 1510, 2446, 6176, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",RL
7,"[0, 14951, 2079, 5331, 12933, 3731, 2522, 1356...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",PS
8,"[0, 3839, 2470, 4191, 2259, 5766, 11187, 12135...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",DP
9,"[0, 19439, 2079, 3782, 2]","[1, 1, 1, 1, 1]",Out


In [96]:
def get_kobert_offset_mappings(kobert_tokens):
    """
    This function is for KoBERTTokenizer, which cannot return offset_mappings.
    """
    kobert_offset_mappings = []
    offset_begin = -1
    for i in kobert_tokens:
        if i in ["[CLS]", "[SEP]"]:
            kobert_offset_mappings.append((0, 0))
            continue
        if i.startswith('▁'):
            offset_begin += 1
            i = i[1:]
        offset_end = offset_begin+len(i)
        kobert_offset_mappings.append((offset_begin, offset_end))
        offset_begin = offset_end

    return kobert_offset_mappings

In [97]:
ne_ids = []

if USE_KOBERT:
    text_tokens = [[cls_token] + tokenizer.tokenize(text)[:max_length-2] + [sep_token] for text in texts]
    offset_mappings = [get_kobert_offset_mappings(text_token) for text_token in text_tokens]
    text_normalize=True
else: 
    offset_mappings = encoded['offset_mapping']
    text_normalize=False

In [98]:
i = 0

for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
    s = text
    t = tokenizer.tokenize(text)
    n = ne
    om = offset_mapping

    print(s)
    print(t)
    print(om)
    print(n)
    print()

    i += 1

    if i == 3:
        break

본적 경북 의성군 의성읍 경북대로 5690-25
['본', '##적', '경북', '의성', '##군', '의성', '##읍', '경북', '##대로', '56', '##90', '-', '25']
[(0, 0), (0, 1), (1, 2), (3, 5), (6, 8), (8, 9), (10, 12), (12, 13), (14, 16), (16, 18), (19, 21), (21, 23), (23, 24), (24, 26), (0, 0)]
{1: {'form': '본적', 'label': 'LC_DOMICILE', 'begin': 0, 'end': 2}, 2: {'form': '경북 의성군 의성읍 경북대로 5690-25', 'label': 'LC_REGION', 'begin': 3, 'end': 26}}

한적한 길을 지나가는 여아를 상대로 나일론 줄로 목을 졸라 실신시킨 후 성추행을 하였고, 심지어는 자신의 처가 교사로 근무하는 초등학교의 학생의 집에 침입하여 부녀자를 희롱하려다 미수에 그치고도 그 후 다시 그 집에 침입하여 여아를 추행하는 등 그 죄질이 아주 좋지 않고,
['한적', '##한', '길', '##을', '지나가', '##는', '여아', '##를', '상대', '##로', '나일', '##론', '줄', '##로', '목', '##을', '졸라', '실', '##신', '##시', '##킨', '후', '성추행', '##을', '하', '##였', '##고', ',', '심지어', '##는', '자신', '##의', '처가', '교사', '##로', '근무', '##하', '##는', '초등', '##학교', '##의', '학생', '##의', '집', '##에', '침입', '##하여', '부녀자', '##를', '희', '##롱', '##하', '##려', '##다', '미수', '##에', '그치', '##고', '##도', '그', '후', '다시', '그', '집', '##에', '침입', '##하여', '여아', '##를', '추행

In [99]:
print(s)
print(t)
print(om)
print(n)

피고인은 위 경찰관들이 위 정우식의 모발 채취를 하지 못하도록 하기 위하여 위 정우식을 막고 서 있는 위 최성욱의 팔을 잡아당기고, 위 김주하의 팔과 상의를 잡아 뒤로 밀어 넘어뜨리고, 그 틈을 이용하여 도주하는 위 정우식을 위 김주하가 뒤따라가자,
['피고인', '##은', '위', '경찰관', '##들이', '위', '정우', '##식', '##의', '모발', '채취', '##를', '하', '##지', '못하', '##도록', '하', '##기', '위하', '##여', '위', '정우', '##식', '##을', '막', '##고', '서', '있', '##는', '위', '최성', '##욱', '##의', '팔', '##을', '잡아당기', '##고', ',', '위', '김주', '##하', '##의', '팔', '##과', '상의', '##를', '잡아', '뒤로', '밀어', '넘어', '##뜨리', '##고', ',', '그', '틈', '##을', '이용', '##하여', '도주', '##하', '##는', '위', '정우', '##식', '##을', '위', '김주', '##하', '##가', '뒤따라', '##가', '##자', ',']
[(0, 0), (0, 3), (3, 4), (5, 6), (7, 10), (10, 12), (13, 14), (15, 17), (17, 18), (18, 19), (20, 22), (23, 25), (25, 26), (27, 28), (28, 29), (30, 32), (32, 34), (35, 36), (36, 37), (38, 40), (40, 41), (42, 43), (44, 46), (46, 47), (47, 48), (49, 50), (50, 51), (52, 53), (54, 55), (55, 56), (57, 58), (59, 61), (61, 62), (62, 63), (64, 65), (65, 66), (67, 71), (71, 72), (72, 73), (74, 75), (76, 7

In [100]:
# for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
#     print(text)
#     print(ne)
#     print(offset_mapping)
#     s = text
#     n = ne
#     om = offset_mapping
#     break

In [101]:
def get_char_labels(sentence, ne, label_to_index, normalize=False):
    """
    sentence: 일본인가 어딘가 간다고
    ne: {1: {'form': '정호성', 'label': 'PS_NAME', 'begin': 7, 'end': 10}, 2: {'form': '비서관', 'label': 'CV_POSITION', 'begin': 11, 'end': 14}}

    Labeling tags by character-level.
    Parameter "normalize" for KoBERTTokenizer, which used to normalize unicode text before tokenizing.
    """
    char_labels = [0] * len(sentence) # sentence의 글자 개수만큼 0으로 채워진 리스트
    # print(sentence) # 그리고 인제 정호성 비서관이
    # print(char_labels) # [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for i in range(1, len(ne)+1): # ne의 인덱스가 1부터 시작하므로 +1 하는 것
        begin = ne[i]['begin'] # 엔티티가 시작하는 문장 인덱스
        end = ne[i]['end'] # 엔티티가 끝나는 문장 인덱스
        print(f"ne: {ne[i]['label']}, {ne[i]['form']}, {ne[i]['begin']}, {ne[i]['end']}")
        # print(begin, end)
        
        B_label = 'B-'+ne[i]['label'][:2] # PS_NAME -> B-PS
        I_label = 'I-'+ne[i]['label'][:2] # PS_NAME -> I-PS
        # print(B_label, I_label)

        print(f'label: {I_label}, {label_to_index[I_label]}')

        for j in range(begin, end):
            char_labels[j] = label_to_index[I_label]
            # char_labels의 begin 부터 end 인덱스까지 I_label에 해당하는 번호로 업데이트 한다.
            # begin = 7, end = 10이고, I_label = I-PS일 때
            # [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]를
            # [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0]로 변형
        # print(char_labels)

        char_labels[begin] = label_to_index[B_label]
        print(char_labels)
        # 그리고 begin에 해당하는 인덱스만 B_label의 번호로 업데이트 한다.
        # [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0]를
        # [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0]로 변경
        # print(char_labels)
    # 이 과정을 각각의 엔티티에 대해서 수행하며 char_labels를 업데이트 해나간다.

    if normalize:
        normalized_sentence = unicodedata.normalize("NFKC", sentence)
        for i, char in enumerate(sentence):
            normalized_char = unicodedata.normalize("NFKC", char)
            if len(normalized_char) > 1:
                char_labels[i:i] = [char_labels[i]] * (len(normalized_char)-1)
                if len(char_labels) == len(normalized_sentence):
                    break        

    return char_labels

In [102]:
print(s)
print(n)

피고인은 위 경찰관들이 위 정우식의 모발 채취를 하지 못하도록 하기 위하여 위 정우식을 막고 서 있는 위 최성욱의 팔을 잡아당기고, 위 김주하의 팔과 상의를 잡아 뒤로 밀어 넘어뜨리고, 그 틈을 이용하여 도주하는 위 정우식을 위 김주하가 뒤따라가자,
{1: {'form': '피고인', 'label': 'PS_ASSAILANT', 'begin': 0, 'end': 3}, 2: {'form': '경찰관', 'label': 'JB', 'begin': 7, 'end': 10}, 3: {'form': '정우식', 'label': 'NM', 'begin': 15, 'end': 18}, 4: {'form': '모발', 'label': 'BP', 'begin': 20, 'end': 22}, 5: {'form': '정우식', 'label': 'NM', 'begin': 44, 'end': 47}, 6: {'form': '최성욱', 'label': 'NM', 'begin': 59, 'end': 62}, 7: {'form': '팔', 'label': 'BP', 'begin': 64, 'end': 65}, 8: {'form': '김주하', 'label': 'NM', 'begin': 76, 'end': 79}, 9: {'form': '팔', 'label': 'BP', 'begin': 81, 'end': 82}, 10: {'form': '정우식', 'label': 'NM', 'begin': 121, 'end': 124}, 11: {'form': '김주하', 'label': 'NM', 'begin': 128, 'end': 131}}


In [103]:
cl = get_char_labels(s, n, label_to_index, False)

ne: PS_ASSAILANT, 피고인, 0, 3
label: I-PS, 18
[17, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ne: JB, 경찰관, 7, 10
label: I-JB, 16
[17, 18, 18, 0, 0, 0, 0, 15, 16, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ne: NM, 정우식, 15, 18
label: I-NM, 10
[17, 18, 18, 0, 0, 0, 0, 15, 16, 16, 0, 

In [104]:
print(cl)

[17, 18, 18, 0, 0, 0, 0, 15, 16, 16, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0, 27, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0, 0, 0, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0]


In [105]:
for i in range(len(s)):
    print(s[i], cl[i], index_to_label[cl[i]])

피 17 B-PS
고 18 I-PS
인 18 I-PS
은 0 O
  0 O
위 0 O
  0 O
경 15 B-JB
찰 16 I-JB
관 16 I-JB
들 0 O
이 0 O
  0 O
위 0 O
  0 O
정 9 B-NM
우 10 I-NM
식 10 I-NM
의 0 O
  0 O
모 27 B-BP
발 28 I-BP
  0 O
채 0 O
취 0 O
를 0 O
  0 O
하 0 O
지 0 O
  0 O
못 0 O
하 0 O
도 0 O
록 0 O
  0 O
하 0 O
기 0 O
  0 O
위 0 O
하 0 O
여 0 O
  0 O
위 0 O
  0 O
정 9 B-NM
우 10 I-NM
식 10 I-NM
을 0 O
  0 O
막 0 O
고 0 O
  0 O
서 0 O
  0 O
있 0 O
는 0 O
  0 O
위 0 O
  0 O
최 9 B-NM
성 10 I-NM
욱 10 I-NM
의 0 O
  0 O
팔 27 B-BP
을 0 O
  0 O
잡 0 O
아 0 O
당 0 O
기 0 O
고 0 O
, 0 O
  0 O
위 0 O
  0 O
김 9 B-NM
주 10 I-NM
하 10 I-NM
의 0 O
  0 O
팔 27 B-BP
과 0 O
  0 O
상 0 O
의 0 O
를 0 O
  0 O
잡 0 O
아 0 O
  0 O
뒤 0 O
로 0 O
  0 O
밀 0 O
어 0 O
  0 O
넘 0 O
어 0 O
뜨 0 O
리 0 O
고 0 O
, 0 O
  0 O
그 0 O
  0 O
틈 0 O
을 0 O
  0 O
이 0 O
용 0 O
하 0 O
여 0 O
  0 O
도 0 O
주 0 O
하 0 O
는 0 O
  0 O
위 0 O
  0 O
정 9 B-NM
우 10 I-NM
식 10 I-NM
을 0 O
  0 O
위 0 O
  0 O
김 9 B-NM
주 10 I-NM
하 10 I-NM
가 0 O
  0 O
뒤 0 O
따 0 O
라 0 O
가 0 O
자 0 O
, 0 O


In [106]:
def get_token_labels(char_labels, offset_mappings):
    """
    char_labels = [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 13, 14, 14, 0]
    offset_mappings = [(0, 0), (0, 3), (4, 6), (7, 9), (9, 10), (11, 14), (14, 15), (0, 0)]
    Labeling tags by token-level.
    """
    token_labels = []

    for offset_mapping in offset_mappings:
        start_offset, end_offset = offset_mapping
        if start_offset == end_offset:# (0, 0) 특수 토큰이면
            token_labels.append(-100) # -100을 리스트에 넣는다.
            continue
        token_labels.append(char_labels[start_offset])
        # print(char_labels[start_offset])

    return token_labels

In [107]:
tl = get_token_labels(cl, om)

In [108]:
print(tl)

[-100, 17, 0, 0, 15, 0, 0, 9, 10, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 0, 0, 0, 0, 0, 0, 0, 9, 10, 0, 27, 0, 0, 0, 0, 0, 9, 10, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 0, 0, 9, 10, 0, 0, 0, 0, 0, -100]


In [109]:
# 맨 앞에 '[CLS]' 추가
t.insert(0, '[CLS]')
# 맨 뒤에 '[SEP]' 추가
t.append('[SEP]')
print(t)

['[CLS]', '피고인', '##은', '위', '경찰관', '##들이', '위', '정우', '##식', '##의', '모발', '채취', '##를', '하', '##지', '못하', '##도록', '하', '##기', '위하', '##여', '위', '정우', '##식', '##을', '막', '##고', '서', '있', '##는', '위', '최성', '##욱', '##의', '팔', '##을', '잡아당기', '##고', ',', '위', '김주', '##하', '##의', '팔', '##과', '상의', '##를', '잡아', '뒤로', '밀어', '넘어', '##뜨리', '##고', ',', '그', '틈', '##을', '이용', '##하여', '도주', '##하', '##는', '위', '정우', '##식', '##을', '위', '김주', '##하', '##가', '뒤따라', '##가', '##자', ',', '[SEP]']


In [110]:
for token, entity_idx in zip(t, tl):
    if entity_idx == -100:
        continue
    else:
        label = index_to_label[entity_idx]
        print(token.ljust(5), label.rjust(5))

피고인    B-PS
##은       O
위         O
경찰관    B-JB
##들이      O
위         O
정우     B-NM
##식    I-NM
##의       O
모발     B-BP
채취        O
##를       O
하         O
##지       O
못하        O
##도록      O
하         O
##기       O
위하        O
##여       O
위         O
정우     B-NM
##식    I-NM
##을       O
막         O
##고       O
서         O
있         O
##는       O
위         O
최성     B-NM
##욱    I-NM
##의       O
팔      B-BP
##을       O
잡아당기      O
##고       O
,         O
위         O
김주     B-NM
##하    I-NM
##의       O
팔      B-BP
##과       O
상의        O
##를       O
잡아        O
뒤로        O
밀어        O
넘어        O
##뜨리      O
##고       O
,         O
그         O
틈         O
##을       O
이용        O
##하여      O
도주        O
##하       O
##는       O
위         O
정우     B-NM
##식    I-NM
##을       O
위         O
김주     B-NM
##하    I-NM
##가       O
뒤따라       O
##가       O
##자       O
,         O


In [111]:
for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
    char_labels = get_char_labels(text, ne, label_to_index, normalize=text_normalize)
    # 오늘 왜 머리 올리고 왔어?
    # [15, 16, 0, 0, 0, 23, 24, 0, 0, 0, 0, 0, 0, 0, 0]
    token_labels = get_token_labels(char_labels, offset_mapping)
    # [-100, 15, 0, 23, 0, 0, 0, 0, 0, -100]
    ne_ids.append(token_labels)
    

print("Sequence labeling completed : |labels| %d" % (len(ne_ids)))

ne: LC_DOMICILE, 본적, 0, 2
label: I-LC, 8
[7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ne: LC_REGION, 경북 의성군 의성읍 경북대로 5690-25, 3, 26
label: I-LC, 8
[7, 8, 0, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
ne: LC_OTHER_FACILITY, 길, 4, 5
label: I-LC, 8
[0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ne: GD, 여아, 12, 14
label: I-GD, 12
[0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [112]:
return_data = pd.DataFrame([encoded["input_ids"], encoded["attention_mask"], ne_ids, data['sentence_class'].values], index=[
                            "input_ids", "attention_mask", "labels", "sentence_class"]).T
if config.with_text:
    return_data['sentence'] = texts

In [113]:
return_data

,input_ids,attention_mask,labels,sentence_class
0,"[0, 1163, 2125, 4724, 15211, 2242, 15211, 2058...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 7, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...",LC
1,"[0, 19243, 2470, 647, 2069, 7129, 2259, 29001,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 7, 0, 0, 0, 11, 0, 0, 0, 33, 34, ...",GD
2,"[0, 9679, 2073, 1485, 9358, 7285, 1485, 12475,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 17, 0, 0, 15, 0, 0, 9, 10, 0, 27, 0, 0,...",JB
3,"[0, 1504, 3863, 7494, 23252, 2357, 22133, 4456...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",QT
4,"[0, 5782, 3900, 17429, 19513, 2460, 19513, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 7, 7, 8, 8, 8, 8, 8, 8, -100]",LC
5,"[0, 717, 18, 20928, 1545, 2248, 2421, 7475, 81...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17,...",PC
6,"[0, 3839, 2470, 5183, 2119, 1510, 2446, 6176, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 9, 10, 10, 17, 0, 0, 0, 0, ...",RL
7,"[0, 14951, 2079, 5331, 12933, 3731, 2522, 1356...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",PS
8,"[0, 3839, 2470, 4191, 2259, 5766, 11187, 12135...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DP
9,"[0, 19439, 2079, 3782, 2]","[1, 1, 1, 1, 1]","[-100, 0, 0, 0, -100]",Out


In [114]:
label_info = {
    "label_list": label_list,
    "label_to_index": label_to_index,
    "index_to_label": index_to_label
}

return_values = {
    "data": return_data.to_dict(),
    "label_info": label_info,
    "pad_token": (tokenizer.pad_token, tokenizer.pad_token_id),
    "pretrained_model_name": tokenizer.name_or_path
}

save_path = config.save_path   
fn = os.path.split(config.load_fn)[1].split('.')[0]
plm_name = pretrained_model_name.replace('/', '_')
save_fn = os.path.join(save_path, f'{fn}.{plm_name}.encoded.pickle')

with open(save_fn, "wb") as f:
    pickle.dump(return_values, f, 4)
print("Encoded data saved as %s " % save_fn)

Encoded data saved as data/encoded/test.klue_roberta-base.encoded.pickle 
